In [3]:
import sys
sys.path.append("..")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import data_frame
import tensorflow as tf
import tensorflow_probability as tfp
import tf_keras
import models
from sklearn.preprocessing import StandardScaler
from scipy.optimize import minimize_scalar

In [5]:
filePath = "../KiDS-DR5-WCScut_x_DESI-DR1-small.fits"

df = data_frame.DataFrame(filePath, "QSO", data_frame.MaxFiller())
X_train, y_train = df.get_train_dataset()
X_test, y_test = df.get_test_dataset()

scaler = StandardScaler()
scaler.fit(X_train)

X_test = scaler.transform(X_test)

In [35]:
config = {"num_components": 3}

all_models = []
for i in range(1, 6):
    model_path = f"../models/experiment2/MG_{config["num_components"]}_components_{i}/variables/variables"
    model = models.MLModelContext(strategy=models.MixtureGaussian(df, config)).strategy.network
    model.load_weights(model_path)
    all_models.append(model)

distributions = []
for model in all_models:
    distributions.append(model(X_test))

In [111]:
dist = distributions[0]
grid = np.arange(-1, 10, 0.01)
eval_grid = []
for point in grid:
    eval_grid.append(dist.log_prob(point).numpy())
eval_grid = np.array(eval_grid).T

In [112]:
modes_idx = np.argmax(eval_grid, axis=1)

In [113]:
grid[modes_idx][:10]

array([1.52, 1.5 , 1.91, 2.47, 2.52, 2.42, 0.86, 2.27, 1.08, 1.98])

In [114]:
df.data[df.data["split"]=="test"]["Z"].min()

0.05165287889774262